In [12]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import numpy as np

from utils import loss
from dataloader import IntentExample
from dataloader import load_intent_examples
from dataloader import sample
from dataloader import InputExample
from dataloader import convert_examples_to_features

In [2]:
!pwd

/home/sitindustry/Documents/Reading/implement_papers


In [3]:
# Few-Shot Intent Detection via Contrastive Pre-Training and Fine-Tuning
# https://github.com/jianguoz/Few-Shot-Intent-Detection

## Todo
 1. load data and combined them from every sources
 2. preprocess -> embedded two sentence into single encoder -> h
 3. use h to calculate contrasive learning loss 
* dont forget to use nn.CosineSimilarity instead of from scartch

## Parameters
- N - number of sentences in the batch 
- t - temperature paramter -> control penalty for neg samples
- u - sentecnce or utterances 
- hi - representation of ui
- hi - BERT(ui) -> we use bert-base-uncased
- i-th -> order of sentence
- hi_bar -> the representation of ui_bar 
- ui_bar -> the same sentence as ui but (10%) tokens are random masked  (Devlin et al.,2019)
- M - the number of maksed tokens in each batch 

## Preprocessesing data
  Todo 
  1. tokenize prepare feature
  2. dont forget to remove utterances that less than five tokens
   * we will all exclude  CLINC-Single-Domain-OOS

## Trainning process
 1. during batch trainning dont forget to randomly maksed, a sentence has 
    different maksed postion across different trannig epochs
 2. (ui ,ui_bar) -> single encoder during batch tranning  (Gao et al., 2021)
      #### detail
      - they take same sentence to encoder twice : by applying the standard          dropout twice -> possivtive pairs
      - take other sentence in mini-batch as negative pairs
      - model -> predict positive one among neg
      - we have to use BERT encoder from (SimCSE) to do standard dropout 
      - the way we make match pos or neg samples to feed encoder (SimCSE)
 3. add mask language modelling loss  (Devlinet al., 2019; Wu et al., 2020a)
 4. P(xm) -> predicted probabilty of maksed token xm over total vocabulary
 
 
 * ref -  (SimCSE: Simple Contrastive Learning of Sentence Embeddings)

## Experiment Setting
- contrasive Pretraining 
 1. Pre-train the combined intent datasets -> combine every dataset (guess)
 2. 15 epochs
 3. batch size = 64
 4. t = 0.1 , lamda = 1.0

- Fine tunning 
 1. 5-shot -> five trainning examples per intents
 2. 10-shot -> tens trainning examples per intents
 3. batch size = 16
 4. t =  {0.1, 0.3, 0.5}
 5. λ ∈ {0.01, 0.03, 0.05}
 6. 30 epochs
 7. apply label smoothing to the intent classifcation loss (Zhang et al. (2020a)

## Load dataset 

In [4]:
!ls ../../dataset/Few-Shot-Intent-Detection/Datasets/CLINC150/oos/train/

label  seq.in


## Config

In [5]:
train_file_path = '../../dataset/Few-Shot-Intent-Detection/Datasets/CLINC150/train/'
N = 100  # number of samples per class (100 full-shot)
T = 1 # number of Trials

In [6]:
train_examples = load_intent_examples(train_file_path)
# structure of this data  [trials] 
# trail -> [dict1,dict2,...,dict#intents]
# every dict -> {'task':'lable name','examples':[text1,text2,..,textN]}
sampled_tasks = [sample(N, train_examples) for i in range(T)]

In [7]:
#(sampled_tasks[0][0]['examples'])

#### summarize make batch procedure
  1. tokenize sentence -> ui
  2. ui orignal , ui_bar -> masked 10% traditional BERT did
  2. use BERT encoder to do standard dropout(SimCSE) to get hi , hi_bar

In [8]:
label_lists = []
intent_train_examples = []

In [9]:
 for i in range(T):
        tasks = sampled_tasks[i]
        label_lists.append([])
        intent_train_examples.append([])
        
        for task in tasks:
            label = task['task']
            examples = task['examples']
            label_lists[-1].append(label)

            for j in range(len(examples)):
                intent_train_examples[-1].append(InputExample(examples[j], None, label))

In [10]:
len(intent_train_examples[0])

15000

In [14]:
train_features, label_distribution = convert_examples_to_features(intent_train_examples[0], train = True)

NameError: name 'self' is not defined

In [15]:
intent_train_examples[0][100].text_a , intent_train_examples[0][100].text_b

('i must transfer ten dollars from my bank of america account to my capital one account',
 None)

In [11]:
# tasks = sampled_tasks[0]
# label = tasks[0]['task']
# label_lists[-1].append(label)

In [12]:
label_lists = [2,3]
label_lists[-1].append(1)

AttributeError: 'int' object has no attribute 'append'

In [ ]:
# https://neptune.ai/blog/how-to-code-bert-using-pytorch-tutorial

## Load Pretrain model from vocabuary

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# Tokenize input 
text = "who was Elon musk ? Elon musk was an entrepreneur and business magnate"
tokenized_text = tokenizer.tokenize(text)

In [ ]:
# Mask a token that we will try to predict with MLM
masked_index = 7
tokenized_text[masked_index] = '[MASK]'
#assert tokenized_text == [who was Elon musk ? Elon musk was an entrepreneur 'and','business','magnate']

In [ ]:
print(len(tokenized_text))
tokenized_text

In [ ]:
# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd 
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

## Load weight Pretrain model

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()
print("eval done")

In [ ]:
# Predict hidden states features for each layer
encoded_layers, _ = model(tokens_tensor, segments_tensors)

In [ ]:
# We have a hidden states for each of the 12 layers in model bert-base-uncased
assert len(encoded_layers) == 12

In [ ]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

# Predict all tokens
predictions = model(tokens_tensor, segments_tensors)

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
#assert predicted_token == 'musk'

In [ ]:
predicted_token

## Config

In [ ]:
N = 10 #None
temperature = 0.5  #None

In [ ]:
a = np.random.rand(3,2)
b = np.random.rand(3,2)

In [ ]:
a.shape , b.shape

In [ ]:
self_supervised_cl(a,b)

In [ ]:
predicted_token